In [ ]:
from transformers import CLIPProcessor, CLIPModel, GPT2LMHeadModel, GPT2Tokenizer
from PIL import Image
import torch

In [ ]:
test_pic = "bear.jpg"

In [ ]:
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

image = Image.open(test_pic)
if image.mode != 'RGB':
    image = image.convert('RGB')
image = image.resize((224, 224))
inputs = clip_processor(images=image, return_tensors="pt")
#print(inputs)

with torch.no_grad():
    image_features = clip_model.get_image_features(**inputs)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [ ]:
file_path = 'descriptions.txt'
objects_corpus = []
with open(file_path, 'r') as file:
    for line in file:
        start = line.find("'") + 1
        end = line.find("'", start)
        items_text = line[start:end]
        items = items_text.split(',')
        items = [item.strip().replace(' ', '_') for item in items]
        objects_corpus.extend(items)
objects_corpus = [item for item in objects_corpus if item != '']
#objects_corpus = ["car", "apple", "computer", "book", "chair","brown-bear"]

text_inputs = clip_processor(text=objects_corpus, return_tensors="pt", padding=True)
with torch.no_grad():
    text_features = clip_model.get_text_features(**text_inputs)

In [ ]:
with torch.no_grad():
    similarity = torch.matmul(text_features, image_features.T)
most_similar_index = similarity.argmax()
most_similar_object = objects_corpus[most_similar_index]
print(most_similar_object)

teddy_bear


In [ ]:
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2")
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

input_text = most_similar_object.replace('_', ' ')
input_ids = gpt2_tokenizer.encode(input_text, return_tensors="pt")

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
gpt2_output = gpt2_model.generate(input_ids, max_length=50)
gpt2_text = gpt2_tokenizer.decode(gpt2_output[0], skip_special_tokens=True)
gpt2_text = gpt2_text.split(".")[1]
print(gpt2_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




The bear was a little bit smaller than the other two, but it was still a little bit bigger than the other two


VisionEncoderDecoderModel

In [ ]:
from transformers import VisionEncoderDecoderModel, ViTFeatureExtractor, GPT2Tokenizer
from PIL import Image
import requests

model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTFeatureExtractor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.61k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/982M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


preprocessor_config.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
from transformers import VisionEncoderDecoderModel, ViTFeatureExtractor, GPT2Tokenizer
from PIL import Image
import requests

model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTFeatureExtractor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
image_url = test_pic

image = Image.open(image_url)

inputs = feature_extractor(images=image, return_tensors="pt")

outputs = model.generate(**inputs)
caption = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(caption)

In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf
from transformers import VisionEncoderDecoderModel, ViTFeatureExtractor, GPT2Tokenizer
from PIL import Image
import requests
# Load a dataset from TensorFlow Datasets
ds, info = tfds.load('coco_captions/2014', split='validation[:20]', with_info=True)

# Prepare the feature extractor and model for inference
def prepare_image(image):
    inputs = feature_extractor(images=image, return_tensors="pt")
    outputs = model.generate(**inputs)
    caption = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return caption

In [ ]:
import os
# Iterate over the dataset
for i, example in enumerate(ds.take(10)):  # taking 5 samples for demonstration
    image = example['image']
    # Convert the tf.Tensor image to a PIL Image
    image_pil = Image.fromarray(image.numpy())

    # Generate a caption
    caption = prepare_image(image_pil)
    # Save the image
    image_filename = os.path.join("/content", f'image_{i}.jpg')
    image_pil.save(image_filename)
    print(f"Saved {image_filename} with caption: {caption}")

Saved /content/image_0.jpg with caption: a double decker bus is parked on the side of the road 
Saved /content/image_1.jpg with caption: a man holding a tennis racquet on a tennis court 
Saved /content/image_2.jpg with caption: a game system with a remote control and a book 
Saved /content/image_3.jpg with caption: a woman walking down a sidewalk next to a sign 
Saved /content/image_4.jpg with caption: a kitchen with a stove, sink, and refrigerator 
Saved /content/image_5.jpg with caption: a stop sign at night with a street light 
Saved /content/image_6.jpg with caption: a baby elephant is playing with a larger elephant 
Saved /content/image_7.jpg with caption: a woman sitting on the ground with a laptop 
Saved /content/image_8.jpg with caption: a table topped with a tray of food and a glass of beer 
Saved /content/image_9.jpg with caption: a number of people on horses on a city street 


evaluation

In [ ]:
pip install git+https://github.com/salaniz/pycocoevalcap.git


  Cloning https://github.com/salaniz/pycocoevalcap.git to /tmp/pip-req-build-c2rkfq11
  Running command git clone --filter=blob:none --quiet https://github.com/salaniz/pycocoevalcap.git /tmp/pip-req-build-c2rkfq11
  Resolved https://github.com/salaniz/pycocoevalcap.git to commit a24f74c408c918f1f4ec34e9514bc8a76ce41ffd
  Preparing metadata (setup.py) ... done


In [ ]:
from pycocoevalcap.cider.cider import Cider
from collections import defaultdict

# Candidate captions (generated by a model)
cand_captions = {
    'id1': ['a double decker bus is parked on the side of the road.'],
    'id2': ['a man holding a tennis racquet on a tennis court.'],
    'id3': ['a game system with a remote control and a book.'],
    'id4': ['a woman walking down a sidewalk next to a sign.'],
    'id5': ['a kitchen with a stove, sink, and refrigerator.'],
    'id6': ['a stop sign at night with a street light.'],
    'id7': ['a baby elephant is playing with a larger elephant.'],
    'id8': ['a woman sitting on the ground with a laptop.'],
    'id9': ['a table topped with a tray of food and a glass of beer.'],
    'id10': ['a number of people on horses on a city street.']
}

# Reference captions (ground truth)
ref_captions = {
    'id1': ["A double decker bus parked by the sidewalk with people nearby."],
    'id2': ["A tennis player in mid-swing on a clay court."],
    'id3': ["A video game system with a remote and game case displayed."],
    'id4': ["A woman walking on a city sidewalk next to a subway sign."],
    'id5': ["A modern kitchen with stainless steel appliances."],
    'id6': ["A stop sign illuminated by a street lamp at night."],
    'id7': ["A young elephant playfully interacting with an adult elephant."],
    'id8': ["A smiling woman sitting on the ground with a laptop."],
    'id9': ["A table with various glasses of beer and an assortment of food."],
    'id10': ["A group of people on horseback waiting at a city intersection."]
}

# Convert reference captions to the format expected by the scorer
gts = ref_captions
# Convert candidate captions to the format expected by the scorer
res = cand_captions
# Initialize the scorer
scorer = Cider()

# Compute CIDEr score
score, _ = scorer.compute_score(res, gts)

print(f"CIDEr Score: {score}")


CIDEr Score: 2.4620395470825454
